In [2]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

In [212]:
import time
import re
import pandas as pd

## Nate news에서 title, publisher, content 가져오기

In [205]:
def get_article(url):
    # Obtain three types of information about a news article
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    
    news_title = re.sub(': 네이트 뉴스','',soup.title.text)
    #publisher = soup.find('meta', attrs={'name':'nate:site_name'}).get('content')
    publisher = soup.find('span',class_='link').text.split('\n')[1]
    news_content = soup.find('div', attrs = {'id':'realArtcContents'}).text
    news_content = re.sub('\n','',news_content.split('@')[0].strip())
    
    return news_title, publisher, news_content

## Selenium으로 검색어 입력한 url 가져오기 (iframe_url)

In [133]:
driver = webdriver.Chrome(executable_path=r'C:\Users\sormd\chromedriver.exe')
driver.get("https://news.nate.com")

elem = driver.find_element_by_id("input_search")
elem.send_keys('AI스피커')
elem.submit()

req = driver.page_source
soup=BeautifulSoup(req, 'html.parser')
iframe = soup.find_all('iframe')
url = iframe[1].get('src')

iframe_url = iframe[1].get('src')
iframe_url = 'http:' + iframe_url

In [148]:
iframe_url

'http://news.search.nate.com/iframe/search.html?q=AI%EC%8A%A4%ED%94%BC%EC%BB%A4'

## iframe_url에서 source code 가져오기

In [140]:
r = requests.get(iframe_url)
soup = BeautifulSoup(r.text.encode('utf-8'), 'html5lib')

# driver.get(iframe_url)
# source = driver.page_source
# soup = BeautifulSoup(source, 'html.parser')

In [100]:
soup.find_all('a')

[<a href="?q=AI%EC%8A%A4%ED%94%BC%EC%BB%A4&amp;o=T">등록일</a>,
 <a href="//news.nate.com/view/20190429n17897" target="_blank"><img alt="뉴스 이미지" onerror="onErrThumbChg(this, 'newsimg_0');" src="//thumbnews.nateimg.co.kr/search/http://news.nateimg.co.kr/orgImg/sg/2019/04/29/20190429503066.jpg"/></a>,
 <a href="//news.nate.com/view/20190429n17897" target="_blank">KT, 화면 달린 <b>AI스피커</b> 선보인다…'내 목소리 동화' 등 개인화...</a>,
 <a class="c-link" href="?q=AI%EC%8A%A4%ED%94%BC%EC%BB%A4&amp;s=sg" title="'세계일보'만 검색">세계일보</a>,
 <a href="//news.nate.com/view/20190428n05933" target="_blank"><img alt="뉴스 이미지" onerror="onErrThumbChg(this, 'newsimg_1');" src="//thumbnews.nateimg.co.kr/search/http://news.nateimg.co.kr/orgImg/kh/2019/04/28/l_2019042801003455000277991.jpg"/></a>,
 <a href="//news.nate.com/view/20190428n05933" target="_blank">2월 문 여는 용인세브란스 병원엔 <b>AI스피커</b>, AR표지판 들어간다</a>,
 <a class="c-link" href="?q=AI%EC%8A%A4%ED%94%BC%EC%BB%A4&amp;s=kh" title="'경향신문'만 검색">경향신문</a>,
 <a href="//news.nate.com/view

## BeautifulSoup으로 페이지마다 news title, url 가져오기

In [226]:
links, titles = [],[]

for page in range(100):
    pages = 10*page + 1
    r = requests.get(iframe_url + '&e=' + str(pages))
    soup = BeautifulSoup(r.text.encode('utf-8'), 'html5lib')


    for i in soup.find_all('a'):
        if 'news' in str(i) and '뉴스 이미지' not in str(i): 
            link = 'http:'+ i.get('href')
            links.append(link)
            title = i.text
            titles.append(title)
            print(link, title)
        

http://news.nate.com/view/20190429n22697 기술력" KT, 스크린 일체형 AI스피커…'키즈' '개인화'로 시장...
http://news.nate.com/view/20190429n17897 KT, 화면 달린 AI스피커 선보인다…'내 목소리 동화' 등 개인화...
http://news.nate.com/view/20190428n05933 2월 문 여는 용인세브란스 병원엔 AI스피커, AR표지판 들어간다
http://news.nate.com/view/20190429n16823 '보이는 AI스피커' 경쟁…KT, 기가지니 테이블TV 공개
http://news.nate.com/view/20190429n10374 KT, 기가지니 확산 '가속'…'보이는 AI스피커' 출시
http://news.nate.com/view/20190429n10381 KT도 보이는 AI스피커 발표…5월2일 출시
http://news.nate.com/view/20190429n10210 보이는 AI스피커, '기가지니 테이블TV' 첫 선…개인화로 간다
http://news.nate.com/view/20190429n23617 귀로 듣고 눈으로 보는 AI 스피커대전…승자는?
http://news.nate.com/view/20190429n17630 목소리로 동화책을"…KT, 스크린 달린 AI 국내 최초 기능(일문일답)
http://news.nate.com/view/20190429n17545 대교 '상상Kids 북클럽', KT '기가지니' 서비스 출시
http://news.nate.com/view/20190429n15593 대교 '상상Kids 북클럽', KT 기가지니 서비스 출시
http://news.nate.com/view/20190429n14744 대교 상상키즈북클럽, 'KT 기가지니' 서비스 출시
http://news.nate.com/view/20190429n07008 연세의료원, 5G 디지털혁신병원 용인에 도입
http://news.nate.com/view/20190429n

http://news.nate.com/view/20190426n12760 SK텔레콤, 누구 개발 공모전 최우수상에 '세줄일기 감성 오디오'
http://news.nate.com/view/20190426n11908 감성 공유 서비스 '세줄일기', SKT '누구'플레이 개발전서 1등
http://news.nate.com/view/20190426n11018 SKT 누구플레이 개발 공모전 최우수상에 '세줄일기 감성 오디오'
http://news.nate.com/view/20190426n09832 SK텔레콤 '누구 플레이 개발 공모전'…'세줄일기 감성 오디오' 최우수상
http://news.nate.com/view/20190426n04796 성대모사 자유자재 AI 유튜브 열풍 올라탔죠
http://news.nate.com/view/20190426n08980 SK텔레콤 '누구 플레이 개발 공모전' 최우수상 수상은 누구?
http://news.nate.com/view/20190426n08776 SKT, 누구플레이 개발 공모전서 '세줄일기 감성 오디오' 최우수상...
http://news.nate.com/view/20190426n08517 SKT, 감성 공유 '세줄일기'에 누구 플레이 최우수상
http://news.nate.com/view/20190426n08360 SK텔레콤, 누구 플레이 개발 공모전서 '세줄일기 감성 오디오' 최우수상...
http://news.nate.com/view/20190426n07942 SKT, '세줄일기 감성 오디오' 누구(NUGU)플레이 개발 공모전'...
http://news.nate.com/view/20190426n07561 '세줄일기 감성 오디오', SKT 누구 플레이 공모전서 최우수상
http://news.nate.com/view/20190426n07040 '세줄일기 감성 오디오', SKT 누구플레이 공모전 최우수상
http://news.nate.com/view/20190425n48106 "인천공항에서 핑크퐁과 아기상어 만나요"
htt

http://news.nate.com/view/20190423n17650 KT-EBS-지니뮤직, 5G 오디오 콘텐츠 제작 MOU
http://news.nate.com/view/20190423n16247 KT, EBS-지니뮤직과 5G 오디오 콘텐츠 제작
http://news.nate.com/view/20190423n16053 KT-EBS-지니뮤직, 5G 오디오 콘텐츠 제작한다
http://news.nate.com/view/20190423n15531 KT·EBS·지니뮤직, 5G 오디오 콘텐츠 제작 맞손
http://news.nate.com/view/20190423n13421 SKT, '월드IT쇼'서 5G 등 혁신서비스 전시
http://news.nate.com/view/20190423n12786 KT, EBS-지니뮤직과 5G 오디오 콘텐츠 제작
http://news.nate.com/view/20190423n12440 KT, EBS-지니뮤직과 5G 오디오 콘텐츠 제작 협업
http://news.nate.com/view/20190423n12062 KT-EBS-지니뮤직, 5G 단말 오디오 콘텐츠 공동 제작 '시동'
http://news.nate.com/view/20190423n10168 KT·EBS·지니뮤직, 5G 오디오 콘텐츠 공동 제작
http://news.nate.com/view/20190423n09980 '교육 서비스에 5G 접목'…KT·LGU+, EBS와 콘텐츠 제작...
http://news.nate.com/view/20190423n09400 EBS, KT-지니뮤직과 5G용 오디오 콘텐츠 제작
http://news.nate.com/view/20190423n08434 SKT, '월드 IT쇼'에 'AR 앵그리버드' 등 5G 서비스 전시
http://news.nate.com/view/20190423n08182 KT-EBS-지니뮤직, '5G 오디오 콘텐츠' 제작한다
http://news.nate.com/view/20190423n08112 KT, EBS-지니뮤직과 5

http://news.nate.com/view/20190311n22367 동두천시, 실버프렌드 AI스피커 지원사업 실시
http://news.nate.com/view/20190311n20697 동두천시 거주 독거노인에게 실버프렌 드 AI스피커 지원
http://news.nate.com/view/20190306n18605 화면 달린 AI스피커가 나온다…라인, 日에서 3월 공개
http://news.nate.com/view/20190305n28551 LG유플러스, 서울시설공단 장애인용 AI스피커 서비스 만들었다
http://news.nate.com/view/20190305n16089 장애인콜택시, AI스피커로'콜'
http://news.nate.com/view/20190305n13900 AI스피커로 콜택시 호출…LGU+, 장애인 편의돕는 AI서비스 출시
http://news.nate.com/view/20190305n05692 서울시설관리공당, AI스피커로 부르는 콜택시 서비스 개시
http://news.nate.com/view/20190305n02723 "장애인 콜택시 불러줘~"…AI스피커 활용 호출 서비스
http://news.nate.com/view/20190209n08726 AI스피커·IoT기기 보급화에 소비자 민원 급증
http://news.nate.com/view/20190209n07403 AI스피커·사물인터넷 민원 급증…작년 30건
http://news.nate.com/view/20190209n02198 AI스피커·IoT기기 보급 늘자 민원도 급증…작년 30건
http://news.nate.com/view/20190208n21292 AI스피커 시장, '폭풍성장' 배경은?
http://news.nate.com/view/20190205n01484 중장년층 '잇 템' 된 AI스피커'
http://news.nate.com/view/20190203n07939 [영상/주리를틀어라] 독자 리퀘스트 - '당신의 AI스피커가 궁금하십니까...
http://news.na

http://news.nate.com/view/20180928n23494 AI스피커 더 똑똑해진 스마트홈 전쟁 "제조사 협업 넓혀라"
http://news.nate.com/view/20180927n31641 LG U+ 척수장애인 돕는 AI스피커 보급
http://news.nate.com/view/20180927n28007 LG유플러스, 척수장애인 가정에 AI스피커 등 IoT 기기 보급
http://news.nate.com/view/20180927n22019 LG유플러스·네이버, 척수장애인 가정에 AI스피커·U+IoT기기 보급
http://news.nate.com/view/20180927n21673 LG유플러스-네이버, 척수장애인 가정에 AI스피커·U+IoT 기기...
http://news.nate.com/view/20180927n11350 LGU+-네이버, 척수장애인 가정에 AI스피커·U+IoT 기기 보급
http://news.nate.com/view/20180927n09552 LGU+, 네이버와 척수장애인 가정에 AI스피커 보급
http://news.nate.com/view/20180927n08883 LGU+·네이버, 척수장애인 가정 300가구에 AI스피커 증정
http://news.nate.com/view/20180927n07674 LG유플러스-네이버, 척수장애인 가정에 AI스피커·U+IoT 기기...
http://news.nate.com/view/20180926n08312 AI스피커 '구글홈' 사용해보니…韓과 비슷한 기능
http://news.nate.com/view/20180923n04639 전세계 가장 많이 팔린 AI스피커 '구글홈' 한국서도 통할까?
http://news.nate.com/view/20180920n30029 LG유플러스, 시각장애인 전용 AI스피커 '소리세상' 서비스 출시
http://news.nate.com/view/20190122n01711 [기업 브리핑] 프리미엄 AI스피커 'LG 엑스붐 AI 씽큐' 출시
http://news.n

http://news.nate.com/view/20181115n39509 자연어처리 적용 AI스피커 특허출원 급증
http://news.nate.com/view/20181115n25649 하나카드, SK텔레콤과 AI스피커 통한 카드서비스 제공 양해각서 체결
http://news.nate.com/view/20180712n22224 AI스피커 이번에는 호텔로 달려가다
http://news.nate.com/view/20181115n22140 하나카드, SKT AI스피커 '누구'로 카드서비스 제공
http://news.nate.com/view/20180712n11557 카날리스, "한국 AI스피커 시장 급성장 올해 세계 5위"
http://news.nate.com/view/20181115n13337 AI스피커 올해 1억대 넘나? 中 알리바바·바이두 '맹추격'
http://news.nate.com/view/20181115n10847 하나카드, SK텔레콤과 AI스피커 통한 카드서비스 제공 양해각서 체결
http://news.nate.com/view/20180712n06993 한국 AI스피커 시장 급성장…"올해 '세계 톱5' 전망"
http://news.nate.com/view/20180711n16906 SK텔레콤 AI스피커 '누구 캔들'출격
http://news.nate.com/view/20181113n22543 美 법원 "AI스피커 녹음정보, 檢에 제공하라"
http://news.nate.com/view/20181112n11141 마인즈랩, AI스피커에서 영어 배우는 'YBM 영어 말하기' 출시
http://news.nate.com/view/20181112n10167 페이스북 AI스피커 '포탈' 유튜브로 동영상 즐긴다
http://news.nate.com/view/20180708n14731 음원차트 흔드는 AI스피커
http://news.nate.com/view/20180708n14196 구글홈, 곧 안방 상륙…'AI스피커 대전' 예고
http://news.nate.c

http://news.nate.com/view/20180817n30687 AI스피커 '누구', 꿀보이스 찾는다
http://news.nate.com/view/20180817n22398 SK텔레콤, AI스피커 목소리 주인공 선발 대회 시작
http://news.nate.com/view/20180817n06936 SKT, AI스피커 'NUGU'에 담길 목소리 주인공 찾는다
http://news.nate.com/view/20180817n06893 SKT AI스피커에 담길 꿀목소리 '누구'?…25일까지 대국민투표
http://news.nate.com/view/20180817n06419 SKT, AI스피커 NUGU 목소리 발굴…'꿀보이스 코리아' 본선
http://news.nate.com/view/20180817n05848 SKT, AI스피커 '누구'에 담길 최고 목소리 발굴 나서
http://news.nate.com/view/20180816n27448 아마존·MS, 구글·애플과 AI스피커 전면전
http://news.nate.com/view/20180816n02503 KT, AI스피커 '기가지니 버디' 출시
http://news.nate.com/view/20180817n07973 SKT, AI스피커 '누구' 목소리 주인공 뽑는다
http://news.nate.com/view/20180815n15699 KT 소형 AI스피커 '기가지니 버디' 출시
http://news.nate.com/view/20180815n14645 KT, 16일 AI스피커 지가지니 버디 출시
http://news.nate.com/view/20180815n10202 KT, 셋톱박스에서 독립한 AI스피커 '기가지니 버디' 출시
http://news.nate.com/view/20180815n08797 KT, 한 손에 잡히는 AI스피커 '기가지니 버디' 내일 출시
http://news.nate.com/view/20180815n07968 손바닥 크기 AI스피커 '기가지니 버디' 출시
http://news.n

http://news.nate.com/view/20180508n14810 SK하이닉스, 독거노인 말벗 AI스피커 '실버프렌드' 제공
http://news.nate.com/view/20180508n01228 SKT AI스피커 누구, 'My Q&A' 이용량 급증
http://news.nate.com/view/20180509n01823 SK하이닉스, 독거노인 2000가구에 AI스피커 지원
http://news.nate.com/view/20180508n32935 SK하이닉스, 독거노인에 AI스피커 '실버프렌드' 무상 제공
http://news.nate.com/view/20180508n14734 SK하이닉스, 독거노인에 AI스피커 '실버프렌드' 제공
http://news.nate.com/view/20180504n02179 "주인 말만 듣는다" 더 똑똑해진 AI스피커
http://news.nate.com/view/20180502n27699 페북도 AI스피커 도전장…해외시장서 '마빈' 출시설
http://news.nate.com/view/20180430n38372 애플 AI스피커 '홈팟' 곧 일본 상륙?…전파인증 통과
http://news.nate.com/view/20180430n15607 네이버, AI스피커 3차 라인업 '프렌즈 미니' 공개
http://news.nate.com/view/20180430n15353 네이버, '작고 가벼운' AI스피커 '프렌즈 미니' 공개
http://news.nate.com/view/20180430n09370 KT AI스피커에 '고마워' 말만 하면 나도 기부천사
http://news.nate.com/view/20180430n29152 네이버 '사과 한알'보다 작고 강력해진 AI스피커 출시
http://news.nate.com/view/20180430n21599 네이버, AI스피커 3차 라인업…'프렌즈 미니' 공개
http://news.nate.com/view/20190420n02033 [화제의 키워드] 보이는 AI 스피커·넷플릭스에 배운다

http://news.nate.com/view/20180811n00403 삼성 '갤럭시홈'으로 AI 스피커 시장 '출사표'…'가격만 낮았어도'
http://news.nate.com/view/20180810n12255 삼성, 항아리 닮은 AI 스피커 '갤럭시홈' 실물 깜짝 공개…11월...
http://news.nate.com/view/20180810n02062 갤노트9 언팩서 깜짝 등장한 AI 스피커 '갤럭시홈' "중요한 연결고리...
http://news.nate.com/view/20180809n10251 [특징주]엔시트론, AI 등 삼성 180조 투자…스피커 부품 공급...
http://news.nate.com/view/20180803n23264 진화하는 AI 스피커시장, 하반기 삼성효과 커질까
http://news.nate.com/view/20180731n29810 실적 숨고르기] 갤럭시노트9 9일 출격…AI 스피커도 조기 등판
http://news.nate.com/view/20180724n12192 SKT·KT,AI스피커로 차 원격 제어 '홈투카' 서비스 출시 잇따라
http://news.nate.com/view/20180724n07752 SK텔레콤·KT, "AI스피커로 시동 걸어요"…적용되는 차종은?
http://news.nate.com/view/20180720n14284 삼성 야심작, AI 스피커 이름은 'MAGBEE'
http://news.nate.com/view/20180720n10401 [단독]삼성 AI 스피커, 이름은 '매그비(MAGBEE)'
http://news.nate.com/view/20180719n38727 삼성전자, 'AI 스피커' 8월 공개…가격은 300달러 전망
http://news.nate.com/view/20180719n12719 달아오르는 AI 스피커 경쟁…아마존·구글 양강 구도에 삼성 출사표
http://news.nate.com/view/20180713n03212 AI 스피커 하반기 2차 대전…신기술 무장 · 영토 확장 '

In [227]:
len(titles), len(links)

(1000, 1000)

# 모든 url의 news title, publisher, content 가져오기  (get_article func)

In [216]:
from tqdm import tqdm_notebook

In [228]:
title, publisher, content = [], [], []

for i in tqdm_notebook(links):
    t, p, c = get_article(i)
    print(t,p)
    
    title.append(t)
    publisher.append(p)
    content.append(c)

"우리만 가능한 기술력" KT, 스크린 일체형 AI스피커…'키즈' '개인화'로 시장 공략  키뉴스
KT, 화면 달린 AI스피커 선보인다…'내 목소리 동화' 등 개인화 서비스 확대  세계일보
내년 2월 문 여는 용인세브란스 병원엔 AI스피커, AR표지판 들어간다  경향신문
'보이는 AI스피커' 경쟁…KT, 기가지니 테이블TV 공개  연합뉴스
KT, 기가지니 확산 '가속'…'보이는 AI스피커' 출시  헤럴드경제
KT도 보이는 AI스피커 발표…5월2일 출시  머니S
보이는 AI스피커, '기가지니 테이블TV' 첫 선…개인화로 간다  이데일리
귀로 듣고 눈으로 보는 AI 스피커대전…승자는?  데일리한국
"아빠 목소리로 동화책을"…KT, 스크린 달린 AI 국내 최초 기능(일문일답)  이데일리
대교 '상상Kids 북클럽', KT '기가지니' 서비스 출시  이데일리
대교 '상상Kids 북클럽', KT 기가지니 서비스 출시  이투데이
대교 상상키즈북클럽, 'KT 기가지니' 서비스 출시  뉴시스
연세의료원, 5G 디지털혁신병원 용인에 도입  이데일리
연세의료원, 5G 디지털혁신병원 용인에 도입  메디컬투데이
5G망 깔린 '디지털혁신병원' 생긴다  파이낸셜뉴스
"TV 끄고 의사선생님 호출해줘", 병실마다 말 알아듣는 AI비서  디지털타임스
'5G 혁신병원' 용인에 탄생…SK텔레콤-연세의료원, 최신 ICT 도입 '맞손'  아주경제
AI 스피커, IPTV 기능까지 탑재…2세대 AI 스피커 본격 경쟁 돌입  조선비즈
[이슈+] '보이는' AI 스피커 경쟁…KT "SKT와 차별점은 IPTV"  한국경제
"우리는 IPTV 나온다"…KT, 보이는 AI 스피커 대전 '맞불'  IT조선
'듣는 AI'에서 '보는 AI'로…이통 3사 AI 스피커 2라운드  중앙일보
KT도 '보이는 AI 스피커' 경쟁 합류  헤럴드경제
KT, 보이는 AI 스피커 나왔다…'기가지니 테이블 TV' 출시  이투데이
[TF현장] "TV에서 아빠 목소리가" KT, '개인용 AI TV' 첫선  더팩트
"AI가 아빠목소리로 동

KT, '5G 오디오' 콘텐츠 만든다  뉴스핌
KT, EBS-지니뮤직과 5G 오디오 콘텐츠 제작한다  쿠키뉴스
KT, EBS-지니뮤직과 5G 오디오 콘텐츠 제작  조선비즈
[WIS 2019]SK텔레콤, 스마트 이노베이션 선보인다  전자신문
SKT·KT, '월드IT쇼 2019'서 5G콘텐츠·기술 전시…'5G이미지 제고 무게'  스포츠서울
한국엠씨엔협회, 성우협회와 손잡고 '보이스 크리에이터' 육성  IT조선
[WIS2019]'월드IT쇼 2019' 개막…5G 이끄는 새로운 세상  전자신문
"당신은 차 안에서 뭘 하고 싶은가?"…상상을 현실로  세계일보
[딜라이트닷넷] "알렉사, 불 꺼줘"…AWS 행사장에서 생긴 일  디지털데일리
KT, EBS-지니뮤직과 5G 오디오 콘텐츠 제작한다  키뉴스
KT, EBS·지니뮤직과 5G 오디오 콘텐츠 제작한다  매일경제
KT, EBS·지니뮤직과 5G향 오디오 콘텐츠 제작 협력  전자신문
SK텔레콤, '월드 IT쇼 2019'서 5G 기술 대거 공개  EBN산업뉴스
KT·LGU+, EBS와 5G 교육 콘텐츠 제작 협업  서울경제
KT, EBS-지니뮤직과 5G 오디오 콘텐츠 제작  디지털데일리
KT, EBS·지니뮤직과 5G 오디오 콘텐츠 제작  파이낸셜뉴스
KT, EBS-지니뮤직과 5G 오디오 콘텐츠 제작  EBN산업뉴스
KT-EBS-지니뮤직, 5G 오디오 콘텐츠 제작 MOU  브릿지경제
KT, EBS-지니뮤직과 5G 오디오 콘텐츠 제작  아이뉴스24
KT-EBS-지니뮤직, 5G 오디오 콘텐츠 제작한다  뉴시스
KT·EBS·지니뮤직, 5G 오디오 콘텐츠 제작 맞손  IT조선
SKT, '월드IT쇼'서 5G 등 혁신서비스 전시  한국경제TV
KT, EBS-지니뮤직과 5G 오디오 콘텐츠 제작  머니투데이
KT, EBS-지니뮤직과 5G 오디오 콘텐츠 제작 협업  뉴스1
KT-EBS-지니뮤직, 5G 단말 오디오 콘텐츠 공동 제작 '시동'  아시아투데이
KT·EBS·지니뮤직, 5G 오디오 콘텐츠 공동 제작  아시아경제
'교육 서비스에 5G 접목'

[SDC 2018-삼성 AI스피커 '갤럭시홈' 첫 선]사용자 따라다니는 음악···"하이" 아침인사땐 거실 전등 ON  서울경제
'소비자를 각인시켜라' 광고로 AI스피커 마케팅 공들이는 인터넷업계  아시아투데이
SKT AI스피커 '누구'에서 '아프리카TV 팟캐스트' 듣는다  뉴스1
LG유플러스, AI스피커에서 음악퀴즈를…'뮤직큐' 출시  이데일리
LGU+, AI스피커가 음악퀴즈 내고 고객이 답하는 '뮤직큐' 출시  머니투데이
LGU+, AI스피커로 음악퀴즈 '큐'  아이뉴스24
LG유플러스, AI스피커 활용 'YBM 영어말하기' 출시  스포츠경향
LGU+, AI스피커 활용 'YBM 영어말하기' 출시  아이뉴스24
AI스피커·번역 앱 위한 외국어 5종 음성DB 배포  매일경제
애플뮤직, 아마존이어 구글 AI스피커로도 듣는다  IT조선
"사투리 못 알아듣는 한국 AI스피커, 정부 규제 때문"  아시아경제
AI스피커 '구글홈' 연동…'인강' 들으며 질문도  머니투데이
구글, 태블릿PC 장착한 AI스피커 국내 출시한다  아주경제
[촌철살IT] "윤선생"→헤이지니, "공룡 들려줘"→핑크퐁…동심 잡은 AI스피커 '누구'  한국경제
AI스피커, 언제 어떤 용도로 많이 쓸까  파이낸셜뉴스
AI스피커와 스마트폰, '스마트글래스' 등장 땐 사라질 유산  아시아경제
[영상/주리를틀어라] 구글홈vs카카오미니vs클로바 중 반응속도가 가장 빠른 스피커는?…AI스피커 퀴즈 대결 2탄  아주경제
앱 만들어 돈 벌 듯, AI스피커로도 돈 번다  아시아경제
[영상/주리를틀어라] 구글홈vs카카오미니vs클로바 AI스피커 퀴즈 대결, 과연 1위는?  아주경제
AI스피커 800만대 시대…민원도 급증  매일경제
[국감현장]대답없는 AI스피커 "사투리를 못 알아들어서…"  뉴스1
AI스피커·IoT기기 관련 민원 급증…"소비자 보호대책 마련 시급"  MBN TV
AI스피커·IoT기기 민원 급증 작년 30건…전년比 6배↑  매일경제
말 안 듣는 AI스피커…관련 민원 '급증'  이데일리
AI스피커·IoT 기기

韓 상륙한 '구글홈', AI스피커 시장 키울카, 삼킬까  스포츠경향
[머니S포토] 구글 AI스피커 답변하는 미키 김 전무  머니S
[머니S포토] 구글 AI스피커 설명하는 미키 김 전무  머니S
[머니S포토] 구글 AI스피커 상륙, 가정에서 즐기는 다양한 서비스  머니S
AI스피커 '구글홈' 국내 상륙…유튜브 앞세워 시장 공략 박차  디지털데일리
NHN벅스, AI스피커 '구글홈' 연동…300회 듣기이용권 제공  뉴스1
AI스피커 '구글홈' 국내 출시…네이버·카카오와 경쟁  문화일보
신세계I&C, AI스피커 '구글홈·구글홈 미니' 총판 돌입  뉴스1
구글, AI스피커 18일 한국 정식 출시…11일 예약판매 시작  뉴시스
[화보]'구글홈' 출시,우리가족 목소리까지 구별하는 AI스피커  아이뉴스24
최대5시간 재생 AI스피커 '카카오 미니C' 판매 시작  문화일보
구글 AI스피커 '구글홈' 한국 상륙…오는 18일 국내 출시  이투데이
2개 언어로 동시에 사용하는 AI스피커 '구글홈' 11일 예판  뉴스1
LG전자, 18일부터 세탁기 등 가전 8종 구글 AI스피커와 연동  머니S
'집안에서 AI스피커로 차량을 제어한다'  연합뉴스
동심 잡아라…AI스피커 키즈콘텐츠 강화  연합뉴스TV
카카오, AI스피커 2탄 '카카오미니C' 정식판매 시작…풀패키지 9만9000원  스포츠서울
[이슈+] 구글·삼성 이름 단 'AI스피커' 출격…흥행 성공할까  한국경제
AI스피커, 레고처럼 내가 직접 만든다  아시아경제
[시승기] AI스피커로 시동 걸고 에어컨 켜고…집안서 車원격조종  매일경제
LGU+, AI스피커 '프렌즈플러스 미니' 10일 출시  뉴스1
[MT리포트] 똑똑해진 'AI스피커', 또 하나의 가족이 되다  머니투데이
네이버, 더 작아지고 똑똑해진 AI스피커 '프렌즈 미니' 5월 2일 정식 발매  조선비즈
구글 AI스피커 '구글홈' 11일 한국 상륙  디지털타임스
구글 AI스피커 '구글홈' 11일 한국 상륙  디지털타임스
구글, AI스피커 '구글홈' 11일 국내 선보인다  연합뉴

삼성전자, AI 스피커 갤럭시홈 첫 전시…시장 경쟁 주목  데일리안
폴더블폰·AI스피커…삼성 미래먹거리 한눈에  헤럴드경제
[키워드포착] 통신사부터 IT기업까지…AI 스피커 '춘추전국시대'  쿠키뉴스
치명적인 증거 찾아내는 '디지털 포렌식'…이젠 IoT·AI스피커로 확장  디지털데일리
"AI스피커로 음악퀴즈 즐기자" LG U+ '뮤직큐' 선봬  파이낸셜뉴스
LGU+, "AI스피커로 음악퀴즈 한 판"  디지털데일리
LG유플러스, AI 스피커 활용한 음악퀴즈 서비스 '뮤직큐' 출시  아시아투데이
LGU+, AI 스피커로 음악퀴즈 서비스…'뮤직큐' 출시  이투데이
"홈팟이랑 똑같네"…'화웨이 AI 스피커' 공개  아시아경제
LG유플러스, "AI 스피커로 원어민 선생님과 대화하세요"  디지털타임스
"포렌식 기술, 드론-클라우드-AI스피커로 확대"  디지털타임스
"모바일 포렌식, 드론·클라우드·AI스피커로 확대"  뉴시스
한컴지엠디, '모바일 포렌식' 기술 확대…드론·클라우드·AI스피커에도 적용  머니투데이
현대백화점, 네이버AI스피커와 음성쇼핑 안내서비스 도입  매일경제
[이슈분석]AI 스피커 보안, 이대로는 위험하다  전자신문
AI 스피커 전쟁, 스마트홈으로 옮겨붙었다  파이낸셜뉴스
네이버-LGU+, 척수장애인 300가구에 AI 스피커 및 IoT 기기 지원  키뉴스
LGU+ 척수장애인협회에 '우리집AI 스피커' 전달  디지털타임스
LGU+-네이버, 척수장애인 가정에 AI 스피커?U+IoT 보급…2년간 요금 면제  아주경제
AI, 스피커에서 디스플레이로 진화  파이낸셜뉴스
"내 말 잘듣는 여자는 AI뿐"…스피커와 사랑에 빠진 꽃할배  중앙일보
"AI 스피커, 시각장애인의 삶에 '빛'이 되다"  OBS TV
'구글 홈' 상륙에…AI 스피커 大戰 재점화  매일경제
[빅데이터] AI스피커야! OO알려줘! 질문1위는?  노컷뉴스
구글홈 국내 상륙…불붙은 AI 스피커 경쟁  국민일보
구글, AI 스피커 공습…춘추전국시대 예고  디지털타임스
두 살 AI 스피커 '누구', 실사용

In [229]:
len(title), len(publisher), len(content), len(links)

(1000, 1000, 1000, 1000)

## dataframe을 csv 파일로 저장

In [224]:
nate_news = pd.DataFrame({'title':title, 'publisher':publisher, 'content':content, 'link':links})
nate_news.head()

,title,publisher,content,link
0,"""우리만 가능한 기술력"" KT, 스크린 일체형 AI스피커…'키즈' '개인화'로 시장...",키뉴스,"개인화 음성합성 기술 P-TTS 적용, '내 목소리 동화' 서비스 제공[키뉴스 석대...",http://news.nate.com/view/20190429n22697
1,"KT, 화면 달린 AI스피커 선보인다…'내 목소리 동화' 등 개인화 서비스 확대",세계일보,“음성 스피커만으로는 정보를 직관적으로 전달하기 어렵습니다. 스크린이 달린 단말을 ...,http://news.nate.com/view/20190429n17897
2,"내년 2월 문 여는 용인세브란스 병원엔 AI스피커, AR표지판 들어간다",경향신문,내년 2월 문을 여는 용인세브란스 병원에 SK텔레콤의 5G기술이 대거 적용된다.SK...,http://news.nate.com/view/20190428n05933
3,"'보이는 AI스피커' 경쟁…KT, 기가지니 테이블TV 공개",연합뉴스,아빠 목소리로 동화 읽어주는 '내 목소리 동화' 서비스 내놔 (서울=연합뉴스) 채새...,http://news.nate.com/view/20190429n16823
4,"KT, 기가지니 확산 '가속'…'보이는 AI스피커' 출시",헤럴드경제,- 내달 2일 ‘기가지니 테이블TV’ 출격…39만6000원- ‘내 목소리 동화’ 등...,http://news.nate.com/view/20190429n10374


In [225]:
nate_news.to_csv('nate_news.csv', encoding='UTF-8')

In [208]:
url = input('url을 입력하세요: ')
url = url
title, publisher, content = get_article(url)

url을 입력하세요: https://news.nate.com/view/20190325n24113


In [209]:
print(title)

LGU+ 마블 AI스피커 출시 


In [210]:
print(publisher)

스포츠동아


In [211]:
print(content)

LG유플러스는 영화 ‘어벤져스4:엔드게임’이 개봉하는 시점에 맞춰 몰입형 디스플레이를 탑재한 차세대 인공지능(AI)스피커 ‘U+AI 어벤져스’를 출시한다고 25일 밝혔다. 좋아하는 마블 캐릭터를 AI스피커 디스플레이 대기화면에 설정해 감상할 수 있으며, 마블 관련 퀴즈 서비스를 통해 캐릭터를 수집할 수 있어 재미를 더한다. LG유플러스가 지난 MWC에서 선보인 디스플레이가 달린 고음질의 원통형 AI스피커로, 네이버의 인공지능 플랫폼 ‘클로바’를 적용했다.스포츠동아 김명근 기자 dionys
